[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AdmiralHonda/ml_intro/blob/main/appendix/logic.ipynb)

In [ ]:
#@title おまじない（単語分割ツールのインストール）
# 下準備です。すみませんが、おまじないだと思ってください
# 日本語や韓国語などのアジア圏の言語における文章を単語に分割するツールのインストールを行っています。

# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null # mecabの利用に必要なライブラリのインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null                    # gitから辞書ファイルのクローン
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1                   # クローンした辞書のインストール
!pip install mecab-python3==0.7 > /dev/null                                                             # 0.7意外だと謎のエラーが発生して安定しないことがある

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc                                                              # 辞書の参照先にインストール先のディレクトリを追加
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"                                                    # mecabの設定ファイルに新しく辞書を追加したことを追記

# **学習と推論**

この章では今までの学習を生かしてアプリの根幹となる部分を構築します。  
具体的には以下の処理を行います。  

- 必要なデータの配置
- 分散表現（単語をベクトル化したもの）の学習
- 分散表現を生かした文章ベクトルの生成
- ユーザーの入力の文章ベクトルへの変換
- 簡単に試してみよう

## **演習環境**

![講習会＿演習環境.png](https://pub-dd9160b14dab4fd08df96674dc1b9692.r2.dev/講習会＿演習環境.png)

簡単単に今見ている演習環境について説明します。  
ここでは上の図のように皆さんのマシンではなく、googleが用意した仮想マシン上で実行し手皆さんはその結果をブラウザで確認するような実行環境になっています。  
そのためデータなどに関してはgoogle driveを用いてやり取りを行う様になります。
ちなみにこのファイルも保存をすると自動的に皆さんのdrive上に保存されます。  

### jupter notebook
jupter notebookというコマンドラインで実行するのではなくてコードと実行環境とワードがすべて同じファイルに詰まったものを使うといったイメージのツールを用います。  
基本的にセルと呼ばれる四角い枠の中にコードや説明書き（マークダウン）を記述します。  
私が今見ているセルにはマークダウンで説明書きをしているセルになります。  
その他にもコードを記述したセルがあります。下で沢山記述しています。実行する際にはセルの左上にある再生ボタンをクリックしてもらえれば実行できます。  

## **使用するデータ**

### **大容量データを扱う工夫**
今回単語の意味を学習するのに使用するデータはwikipediaの本文すべてです。  
そのデータはテキストファイルではありますが、3.5Gもあります。  
これだけ大きくなるとGoogleDriveでダウンロードする際にウイルスチェックができないので確認画面が表示されます。  
そうするとプログラムから呼び出した際には、確認画面のhtmlが入力され、データ本体は読み込めなくなります。  
#### **Colabのアップロードの遅さ**
またこのColab環境では無料で使える代わりに、データのアップロードがとても遅く、これほどの規模のデータとなると、いったんローカルにダウンロードしてからアップロードすると何時間もかかってしまいます。  
そのため、今起動しているcolabの仮想マシンに皆さんのGoogleDriveをマウント（別のマシンのディレクトリを今のマシンでも扱えるようにする）することで大容量のデータのやり取りを行います。  
正直それでも遅いですが、まだ現実的な時間で実行することができます。  
#### **Driveへデータを配置する**
まずは皆さんのGoogleDriveに下記のデータを配置してください。  
注意として、今のアカウントの保存容量を確認してください。  
今回の演習では約5Gのデータをダウンロードするので、空き容量がない場合は**購入**か**データの削除**が必要です。  
ただ、対面でのword2vecの学習は数時間かかるので行いません。なので今回は学習済みデータを配るので、最低1.5Gくらいの空きがあれば対応できます。  
また、日大のアカウントでは容量無限大なので日大の在校生の場合は学校のアカウントで行うといいでしょう。  
ほかのデータと区別できるように皆さんのdriveの一番上の階層に`/python_ml_intro`というディレクトリを作成し、その中に下記のデータを配置してください。  

>[wikiの学習データ](https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/data/class_select_app/wiki_wakati.txt)  
対面の講義ではおそらく使用しませんが、宿題や確認で学習する際に使用してください。  

>[wikiから学習した学習済みデータ](https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/data/class_select_app/wiki_test_vec.pt)  
対面でプレゼンをする際は学習する時間がもったいないので既に学習済みのものを使用します。  


## ドライブのマウント解説
`drive.mount("/content/drive")`についてですが、あなたのGoogleDriveのルートディレクトリを`/content/drive`として今の実行環境では扱うようにしています。  
現在Colabで実行しているディレクトリは`/content`直下になるのでその下に`drive`というディレクトリを作成していることになります。  
パソコンで見ている場合は左側にあるファルダマークをクリックしてみるとよいでしょう。  
今の作業ディレクトリの下にあるツリーが見れます。  
また`/content/drive`の下に`/My Drive`というディレクトリがありますが、その下にあなたのGoogleDriveのマイドライブの中身が入っているはずです。  
共有フォルダは認識されなかったと思います。  
この先、先ほど紹介した通りにファイルを配置した際には`/content/drive/My Drive/python_ml_intro`の後にファイル名を記述すれば参照できます。  

In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## **word2vecの学習**  

下記のコードの解説をします。

### **学習するためのデータ型に**  

学習するために単語が事前にスペースで区切られたテキストファイルを用意しましたが、さらに改行`\n`ごとに分割して適切な学習単位にする必要があります。  
前処理にて１記事毎や１コンテンツ毎に改行を入れていたのはこのためです。  
`word2vec`のメソッドである`LineSentence`でその処理を行います。  
引数に分割するテキストファイルのパスを渡します。  

### **学習**  

学習する際にテキストデータを改行で区切った`sentences`を引数にとるのはもちろんのこと、その他にもいくつか指定しています。それらについて触れます。  
- iter  
試行回数。何回学習データで学習するかの回数です。何回問題集を解くのかだと思えばええです。
- size  
入力層の列数。
- sg  
cbowかskip-gramかの選択。1か0で指定する。
- min_count  
学習データ中にあまり頻出しない単語を排除するために、出現回数が`min_count`以内であったら学習対象から除外するという設定。不必要な学習単語数の増加は重み（行列）の行数が増えるので計算時間が増えます。また、最終的にデータの量が多くなりアプリで使用する際に読み込み時間が増加する懸念があります。  私の実装ではコンテナを使用することが前提なので数秒の差は大きく見ています。
- window  
周辺の単語をどれだけ考慮するかの数。windowサイズです。
- workers  
実行するプロセスの数です。環境の論理cpu数を指定するとよいでしょう
- hs  
損失計算の際に階層的ソフトマックスかネガティブサンプリングを選ぶかの指定。1か0で指定。

### **保存**

最後の一文では入力層の部分のみを取り出して保存しています。  
`Word2Vec`クラスのパラメータである`wv`が入力層に当たります。  
保存するメソッドの`binary=True`はテキストでなくバイナリで保存するように指定しています。  


In [ ]:
"""
とりあえず学習してみる
この設定だと、２時間近くかかります。
"""
from gensim.models import word2vec

sentences = word2vec.LineSentence("/content/drive/MyDrive/python_ml_intro/wiki_wakati.txt")
model = word2vec.Word2Vec(sentences, iter=5,size=300,sg=1,min_count=5, window=3, workers=4,hs=0)
model.wv.save_word2vec_format("/content/drive/MyDrive/python_ml_intro/wiki_test_vec.pt",binary=True)

## 学習データの読み込み

前述したコードで.pt(区別のために適当につけた拡張子)とついたファイルを単語の分散表現として保存しました。  
今度はそれを`KeyVectors`として読み込みます。学習はできませんが、単語間の**意味**的な類似度や特定の単語のベクトルを抜き出すことができます。  

In [ ]:
from gensim.models import KeyedVectors

wiki_model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/python_ml_intro/wiki_test_vec.pt",binary=True)

## コンテンツの読み込み

これ以降の処理ではwikipediaから学んだ単語の意味を用いてコンテンツの意味を含んだベクトルを生成します。  

今回はサンプルとして授業の検索アプリを作成します。  
少ないですが、20件の講義データをcsv形式のファイルといて用意しています。  
これを読み込み、各講義データごとに所属する単語ベクトルの平均をとり、文章ベクトルを生成します。  

### pandas

pythonのライブラリで表形式のデータを処理するのに非常に便利です。  
この３年生向けのイントロでは触れませんが、データからより学習するために不必要な記号を削除したり改行文字を取り払うなどの前処理をするときなどにも役に立ちます。  
表のデータを列ごとに分析したり、データベースのように検索することもできます。


In [ ]:
import pandas as pd

classroom_info = pd.read_csv("https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/data/class_select_app/class_info.csv")  # データの読み込み
classroom_info.head(10)                                                                                                         # 先頭４個分のデータを確認

In [ ]:
classroom_info_dict = classroom_info.to_dict(orient='record')                                                                   # 読み込んだデータを辞書型に変換
print(classroom_info_dict)

[{'教科名': 'プログラミングの基礎及び演習', '担当者': '和泉 勇治', '授業目的': 'プログラミングの基礎及び演習は，ソフトウェアにより実現する機能を分析し，処理手順を記述し，プログラミング言語の基本文法を用い，構造化されたプログラムの作成の理解にある。', '教育目標': '(1) C言語の基本文法を理解し，分岐や繰返しからなる基本的な命令をC言語で記述する方法を理解する。更に，配列や構造体および機能的な処理をまとめる関数作成の考えを理解し，配列や構造体を扱う関数を用いたより実用的な形式のプログラムをC言語で記述する方法を理解する。(2) 与えられた課題を分析し，3つの基本制御構造（順次，分岐，繰り返し）を適切に組み合わせ，フローチャートなどにより処理手順を図式化して記述できる。加えて，既述した処理手順をC言語の命令に対応させ，プログラムを作成できる。(3) 自主的学修の成果となる課題レポートを毎回提出することにより，自主的な学修を継続することができる。', '概要': '講義において，分岐，繰返し，関数，配列，構造体といったＣ言語によるプログラミングについてソースコード例を示しながら解説する。講義に続き行われる演習においては，第1回から第5回，第7回から第11回，第13回と第14回に出題する課題について，問題分析から処理手順の図式化およびプログラムの実装・確認までを実施する。演習において，教員とTAが適宜指導を行うことにより，演習内容における理解の向上を図る。演習で作成したソースコードは課題提出システムにより正しく実行できるかを各人で確認できる。なお，日々の自主学修の目標となるように，中間試験を２回実施し，試験終了後に解説を行い理解の定着を図る。', '成績評価': '評価は絶対評価とし，授業内試験を50%，中間試験を30%，演習課題およびレポートを20%，これらを統合して100点満点とし，60点以上を合格とする。全てのレポートを提出しない場合には，授業内試験の受験資格を失う。期末試験において理解度が不十分であると考えられる場合は，再試験を行うことがある。', '課題': '演習において，教員およびTAにより，正しい設計方法を指導し，作成したソースプログラムは正しい設計方法，コーディング方法，エラーの分析方法を指導し，誤りを指摘し修正方法を指導

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.


In [ ]:
#@title おまじないのセットアップ
# 文章を単語毎にスペースで区切ってくれるライブラリ
# Mecabのセッティング
import MeCab

tokenizer = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati")

In [ ]:
print(tokenizer.parse("学術どうせ参考になるコメント来ないのにめんどくせーなー")[:-1])

## コンテンツのベクトル化



In [ ]:
"""
ここでは各講義の
・授業目的
・教育目標
・概要
の三つの文章を連結し、mecabによって
単語の集合に分割してから各単語ベクトルの平均を求めています。
"""
import numpy as np

class_content = []                                # コンテンツのベクトルを格納するリスト

for classroom in classroom_info_dict:             # classroom_info_dictには各授業の概要や教科名などが辞書型になったものがリストとして格納されている。
  tmp = ""
  tmp += classroom["授業目的"]
  tmp += classroom["教育目標"]
  tmp += classroom["概要"]

  sum = np.zeros(300)                             # 授業の文章ベクトルを格納する要素を0とした要素数300ノベクトルを初期化。
  words = tokenizer.parse(tmp)[:-1].split(" ")    # 集約した文章を単語のリストに分割。mecabで分割した際には単語間にスペースが入った文字列として出力。最後の改行は邪魔なので考慮していない。
  recg_word_num = 0                               # 文章内で認識できた単語の数を数える
  for word in words:
    try:
      sum += wiki_model[word]
      recg_word_num += 1
    except KeyError:                              # 学習していない単語の場合は考慮しない
      pass
  if recg_word_num == 0:                          # もし学習済みの単語がない場合はランダムなベクトルを割り当てる
    class_content.append(np.random(300))
  else:
    class_content.append(sum/recg_word_num)

In [ ]:
import json

class_content = np.array(class_content)                                                                   # 保存のためにコンテンツベクトルを集約したものをnumpy配列とする
np.save("/content/drive/MyDrive/python_ml_intro/content_vec.npy",class_content)

output_content_info = json.dumps(classroom_info_dict,ensure_ascii=False,indent=2)                         # ユーザーに表示するデータとして保存。コンテンツベクトルと添え字を合わせる
with open("/content/drive/MyDrive/python_ml_intro/content_info_dict.json","w") as f:
  f.write(output_content_info)

## **検索してみよう**
せっかくなので検索して今までの成果を確認してみましょう。  
文章を入力してそれに近い授業を見つけるのです。

### **コンテンツと比較してみよう**

いろいろ準備してきましたが、まだ検索はできません。ユーザーの入力はコンテンツのベクトル化と同じやり方で実装できますが、肝心の比較する部分に関して紹介していません。もう少し頑張りましょう。  


In [ ]:
"""
ユーザーの入力をベクトルとして返す関数の定義
後でwebアプリ化する際に使用します。
"""

def user_input(query: str,m: MeCab.Tagger,wv: KeyedVectors) -> np.ndarray:
  sum = np.zeros(300)                               # 授業の文章ベクトルを格納する要素を0とした要素数300ノベクトルを初期化。
  words = tokenizer.parse(query)[:-1].split(" ")    # 集約した文章を単語のリストに分割。mecabで分割した際には単語間にスペースが入った文字列として出力。最後の改行は邪魔なので考慮していない。
  recg_word_num = 0                                 # 文章内で認識できた単語の数を数える
  for word in words:
    try:
      sum += wv[word]
      recg_word_num += 1
    except KeyError:                                # 学習していない単語の場合は考慮しない
      pass
  if recg_word_num == 0:                            # もし学習済みの単語がない場合はランダムなベクトルを割り当てる
    return np.random(300)
  else:
    return sum / recg_word_num

## コンテンツとの比較

コンテンツとの比較とはベクトル同士の比較です。  
今回はコサイン尺度を紹介します。その他にもユークリッド距離やピアソン相関などがあります。   
コサイン尺度では２つのベクトルΑ,Βがあった時の類似度Tは以下の式で算出されます。  

$$Τ = \frac{Α·Β}{|Α|·|Β|}$$  

高校のcosineを座標から求めた時を思い出しましょう。  
ベクトルを２次元座標で考えたときに同じであればそれぞれの点と点の角度は0になり(点と点との間に差がない)、まったく違うなら90°,180°になります。全く違うときにそのベクトル同士は直行するといいましたね。その時cosineで0°は１、90°は0になりましたね。0から-1までの数字をとるので、0に近ければ全く違う、1なら同じ、-1なら逆のコンテンツという解釈もできます。これを今回は比較する際の計算式とします。


In [ ]:
"""
ユーザーの入力（ベクトル）との比較を行う関数
ユーザーの入力と最も類似度が高いコンテンツの添え字を返す
"""
from sklearn.metrics.pairwise import cosine_similarity
def culculate_sim(query :np.ndarray ,content_vec :np.ndarray) -> int:
  sim_rate = cosine_similarity([query],content_vec)
  return np.argmax(sim_rate)

In [ ]:
user_query = "キャリアプラン"
user_vec = user_input(user_query,tokenizer,wiki_model)
print(classroom_info_dict[culculate_sim(user_vec,class_content)])

## 検索結果に満足できたかな？

やったね。アプリ化するときの基礎が出来上がったZOY  
でも検索結果に満足できたかな？予期しない答えはなかったですか？  

### すべてを考慮してはならない

何がいけなかったのか。学習データがいけないのでしょうのか（実際辞書であるwikiから学習したモデルに、話し言葉を突っ込むことに違和感はあります）  
簡単な解決法としては考慮する単語を減らすことです。文章を読むときに全てにしっかり目を通すでしょうか？  
実際は特徴のある単語に目をつけてその単語間のつながりを見ているとも言えます。  
例を挙げると「昨日 、 友達 と もつ焼き の 店 に 行 った 。」という文章があった時に「友達」や「行 った」という単語はどの話し言葉にも文章にも表れますが、「もつ焼き」という単語はどの文書にも出てくるとは言えないでしょう。なのでこの文章は「もつ焼き」を中心として成り立っているといえます。  
特に今回は文章の特徴を際立たせる必要があるので、文章内で特徴のある単語ベクトルが文章ベクトルに大きく寄与していることが望ましいです。  
以下では文章内の単語がどれだけ重要なのか、そしてどれくらい重要なのかを数値で算出するTF-IDFという手法を紹介します。　　

### TF-IDF

この手法では  
- 単語がその文章内でどれだけ出現しているか
- 学習するデータ内でどれだけその単語が出現しているか
で判断します。　　

#### TF(Term Frequency)

文章内でどれだけその単語が出現しているかを算出する式です。
対象の単語をt、単語が所属する文章をd、学習データ内にある任意の単語をcとすると:
$$TF(t,d) = \frac{num(t)}{\Sigma  num(c) (c \in d)}$$
となります。文章に含まれる単語のうち、どれだけその単語が占めているかを示しています。  

#### IDF(Inverse Document Frequency)

学習データ内でどれだけその単語が出現しているかを示します。  
TFと違ってこちらは希少であるほうが良いので出現回数が小さいほど数値が大きくなるような式になっています。  
対象の単語をt、単語が所属する文章数をdとし、全文章数をnすると:
$$IDF(t,d) = \log \frac{n}{d(t \in d)} $$
となります。対数をとっているのは文章数が膨大になっても計算可能な範囲に留めておくためです。

以上２つの指標を掛けたものが単語がどれだけ重要かを示し指標であるTF-IDFです。  

$$TFIDF = TF \cdot IDF$$

これで単語の重要度を数値で表すことができました。  
今度はこれを使って文章ベクトルを作る際に各単語ベクトルに重みとしてかけることで接続詞や助詞などのよく出る単語を低くし、固有名詞などの特徴ある単語を際立たせてみましょう。

In [ ]:
# tf-idfを計算するコードだが、メモリが64G以上ない場合はやめておきましょう
from gensim import corpora
from gensim.models import TfidfModel

dic = corpora.Dictionary(corpus)
dic.save_as_text("./wiki_dic.dic",sort_by_word=True)
input_corpus = list(map(dic.doc2bow,corpus))                                                                                            # corpusは２次元配列で、各文章を単語のリストとして格納
test_model = TfidfModel(input_corpus)
test_model.save("./wiki_tfidf.model")

In [ ]:
from gensim import corpora
from gensim.models import TfidfModel

dic = corpora.Dictionary.load_from_text("https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/data/class_select_app/wiki_dic.dic")
test_model = TfidfModel().load("https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/data/class_select_app/wiki_tfidf.model")

"""
ここでは各文章毎に属している単語がどれだけ重要なものであるかを重みづけしていきます。
"""

tmp_content_split = [ tokenizer.parse(classroom["授業目的"] + classroom["教育目標"] + classroom["概要"]).replace("\n","").split(" ") for classroom in classroom_info_dict]
input_corpus = list(map(dic.doc2bow,tmp_content_split))  
content_tfidf = test_model[input_corpus]

In [ ]:
"""
ここでは各講義の
・授業目的
・教育目標
・概要
の三つの文章を連結し、mecabによって
単語の集合に分割してから各単語ベクトルにtfidfで得た重みを掛けたベクトルの加重平均を求めています。
"""
import numpy as np

class_content = []                                # コンテンツのベクトルを格納するリスト

for content in content_tfidf:

  sum = np.zeros(300)                             # 授業の文章ベクトルを格納する要素を0とした要素数300ノベクトルを初期化。
  recg_word_num = 0                               # 文章内で認識できた単語だけtfidfの重みを加算する
  for word in content:
    try:
      sum += wiki_model[dic[word[0]]] * word[1]
      recg_word_num += word[1]
    except KeyError:                              # 学習していない単語の場合は考慮しない
      pass
      
  if recg_word_num == 0:                          # もし学習済みの単語がない場合はランダムなベクトルを割り当てる
    class_content.append(np.random(300))
  else:
    class_content.append(sum/recg_word_num)

In [ ]:
user_query = "キャリアプラン"
user_vec = user_input(user_query,tokenizer,wiki_model)
print(classroom_info_dict[culculate_sim(user_vec,class_content)])